<picture> <img align="center" src="https://hermes.digitalinnovation.one/assets/diome/logo-full.svg" width = 150px></picture>

**Autor:** RAFAEL ARRUDA 

**Profissão:** Data Analyst

**Bootcamp:** Santander Bootcamp 2023 - Ciência de Dados com Python

**Projeto:** Explorando IA Generativa em um Pipeline de ETL com Python

**Introdução**

O foco deste projeto é aprimorar as habilidades relacionadas à IA Generativa, incluindo o chatbot GPT, com o objetivo de alcançar resultados mais precisos e alinhados com as metas estabelecidas. Uma das bases fundamentais desse projeto é a aplicação de técnicas de ETL (Extração, Transformação e Carregamento). No campo da ciência de dados, o processo ETL desempenha um papel crucial ao coletar informações de uma ou várias fontes, realizar uma limpeza detalhada, transformar os dados de acordo com as necessidades e, por fim, carregar esses dados para seus destinos finais.

**Metodologia do Projeto**

Seguindo o projeto incial, foi usado o chat GPT-3.5 Turbo.

O trabalho é consumir o endpoint GET https://sdw-2023-prd.up.railway.app/users/{id} (API da Santander Dev Week 2023) para obter os dados de cada cliente.
Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint PUT https://sdw-2023-prd.up.railway.app/users/{id}.


A partir deste ponto, começa o processo de ETL propriamente dito.

- **Extração:** Base coletada a partir do endpint/ .csv.
- **Transformação:**  Tratamento foram executadas. Para a geração de mensagens utilizando a IA Generativa, 
- **Carregamento:** Transformação das alterações em .csv.

# Extração dos dados

**Criação de Tabelas .CSV

Foram criados os USERS  as tabelas no contexto de um banco.

Da seguinte forma https://sdw-2023-prd.up.railway.app/ :

1. Criaremos a tabela de cadastro de usuários com os seguintes atributos:
   - ID
   - Nome
   - CONTA
   - CARD
   - SALDO 
   


2. Criar mensagens enviadas com CHATGPT com os seguintes atributos:
   - ID (Referente ao usuário)
   - # (Referente ao usuário)
   - NEWS Anúncio (inicialmente vazio)



In [1]:
# Utilize sua própria URL se quiser ;)
# Repositório da API: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

# Extract Data

In [10]:

import pandas as pd

df = pd.read_csv('santander-dados.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

[822, 823, 824]


In [ ]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

In [ ]:
[
  {
    "id": 822,
    "name": "bbabyshark",
    "account": {
      "id": 858,
      "number": "666777-1",
      "agency": "0001",
      "balance": 0.0,
      "limit": 0.0
    },
    "card": {
      "id": 794,
      "number": "1234 1234 45678 1111",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 823,
    "name": "Pypy",
    "account": {
      "id": 859,
      "number": "666777-2",
      "agency": "0001",
      "balance": 0.0,
      "limit": 0.0
    },
    "card": {
      "id": 795,
      "number": "1234 1234 45678 2222",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  },
  {
    "id": 824,
    "name": "Peterson",
    "account": {
      "id": 860,
      "number": "666777-3",
      "agency": "0001",
      "balance": 0.0,
      "limit": 0.0
    },
    "card": {
      "id": 796,
      "number": "1234 1234 45678 3333",
      "limit": 1000.0
    },
    "features": [],
    "news": []
  }
]

Transform
- Utilizando a API do OpenAI GPT-4 para gerar uma mensagem de marketing personalizada para cada usuário.

In [ ]:
!pip install openai

In [11]:
# Documentação Oficial da API OpenAI: https://platform.openai.com/docs/api-reference/introduction
# Informações sobre o Período Gratuito: https://help.openai.com/en/articles/4936830

# Para gerar uma API Key:
# 1. Crie uma conta na OpenAI
# 2. Acesse a seção "API Keys"
# 3. Clique em "Create API Key"
# Link direto: https://platform.openai.com/account/api-keys

# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
openai_api_key = '*********************************************************'

In [ ]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos no Santander (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

In [ ]:
Invista no Santander e garanta seu futuro. #BBabySharkReturns
Invista no Santander e garanta seu futuro financeiro.
Invista no Santander e garanta um futuro financeiro sólido.

# Carregando os Dados
-Atualizando a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [8]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")

In [9]:
User bbabyshark updated? True!
User Pypy updated? True!
User Peterson updated? True!

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>